In [7]:
import pandas as pd
from openpyxl import load_workbook 
from pathlib import Path

In [8]:
def main():
    run_file_loop()

In [10]:
def run_file_loop():
    folder = get_files_folder()
    for file in folder:
        hospital_location = get_hospital_name(file)
        processed_file = transform_data(file,hospital_location)
        add_sheet_to_outfile(processed_file,hospital_location)


def get_files_folder() -> list:
    input_folder = (Path.cwd()/'data'/'raw').glob('*.*')
    return [file for file in input_folder if file.is_file()]


def get_hospital_name(file_name) -> str:
    return str(file_name).split("\\")[-1].split()[0]


def add_sheet_to_outfile(dataframe, hospital_location):
    fn = Path.cwd()/'data'/'processed'/'template2.xlsx'
    writer = pd.ExcelWriter(fn, engine='openpyxl', mode='a')
    book = load_workbook(fn)
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
    dataframe.to_excel(writer, sheet_name= hospital_location, index=False)
    writer.save()


def transform_data(file_name,hospital_location) -> pd.DataFrame:
    # read hospital Avimark file
    df = pd.read_excel(file_name, header=None)
    # if there aren't inactive client in file there are 13 columns otherwise 15 columns, add 2 more empty columns if inactive case
    if len(df.columns) != 15:
        df = df.reindex(columns=range(15))
    # remove blank columns and Phone column
    df = df.drop([2,3,4], axis=1)
    # remove first four rows
    df = df.drop([0,1,2,3])
    # get list of index columns prepare for swap columns [0, 1, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
    titles = list(df.columns)
    # swap columns pair (Name<->ClientId)
    titles[0], titles[1] = titles[1], titles[0]
    # swap columns pair (balance<->NAN column)
    titles[3], titles[-2] = titles[-2], titles[3]
    # swap columns pair (Last<->NAN column)
    titles[4], titles[-1] = titles[-1], titles[4]
    # dataframe with swap columns
    df = df[titles]
    # remove balance and payment columns with index 5 , 7
    df = df.drop([5,7],axis=1)
    # insert STATUS column last place
    df.loc[:, "STATUS"] = "ACTIVE"
    #reset columns index
    df = df.reset_index(drop=True)
    # rename row 1 to be header
    df.iloc[0] = ['Name', 'Client ID', 'DR/', 'Location', '0-30 Days', '31-60 Days', '61-90 Days', '90+ Days', 'Total', 'Last Payment Date'
    ,'STATUS']
    # assign header name to variable new header
    new_header = df.iloc[0]
    # dataframe start from blank row
    df = df[1:]
    # rename columns
    df.columns = new_header
    # check if there are inactive clients in report and return index of row in list
    inactive_location = df.index[df.loc[:,'Last Payment Date']=='Inactive Client Balances']
    #Check if inactive client exist, change status to inactive
    if len (inactive_location) != 0:
        df.loc[inactive_location[0]:,'STATUS'] = "INACTIVE"
    # remove blank row
    df.dropna(subset = ["Name"], inplace=True)
    # remove sub header in credit balance and inactive balance
    df = df[df['0-30 Days']!="Current"]
    # change credit text amount to negative number in columns "0-30 Days","31-60 Days","61-90 Days","90+ Days" and "Total"
    df['check_30'] = df["0-30 Days"].apply(
               lambda x: (-1 if 'CR' in x else 1))
    df['check_31_60'] = df["31-60 Days"].apply(
               lambda x: (-1 if 'CR' in x else 1))
    df['check_61_90'] = df["61-90 Days"].apply(
               lambda x: (-1 if 'CR' in x else 1))
    df['check_90'] = df["90+ Days"].apply(
               lambda x: (-1 if 'CR' in x else 1))
    df['check_total'] = df["Total"].apply(
               lambda x: (-1 if 'CR' in x else 1))
    df["0-30 Days"] = df["0-30 Days"].map(lambda x: x.replace('CR',''))
    df["31-60 Days"] = df["31-60 Days"].map(lambda x: x.replace('CR',''))
    df["61-90 Days"] = df["61-90 Days"].map(lambda x: x.replace('CR',''))
    df["90+ Days"] = df["90+ Days"].map(lambda x: x.replace('CR',''))
    df["Total"] = df["Total"].map(lambda x: x.replace('CR',''))
    # change type of columns to float type
    df[['0-30 Days', '31-60 Days', '61-90 Days', '90+ Days', 'Total']] = df[['0-30 Days', '31-60 Days', '61-90 Days', '90+ Days', 'Total']].astype(float)
    df["0-30 Days"] = df["0-30 Days"] * df['check_30']
    df["31-60 Days"] = df["31-60 Days"] * df['check_31_60']
    df["61-90 Days"] = df["61-90 Days"] * df['check_61_90']
    df["90+ Days"] = df["90+ Days"] * df['check_90']
    df["Total"] = df["Total"] * df['check_total']
    df["90+ Days"] = df["90+ Days"] + df["Total"]
    df["Total"] = df["0-30 Days"] + df["31-60 Days"] + df["61-90 Days"] + df["90+ Days"]
    # drop sub check columns 1 and -1
    df = df.drop(["check_30","check_31_60","check_61_90","check_90","check_total"], axis=1)
    df['DR/'] = df["Total"].apply(lambda x: ("DR" if x>= 0 else "CR"))
    #fill hospital name
    df['Location'] = hospital_location
    #check result total
    return df



In [11]:
main()
